In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import spacy
spacy.load('en_core_web_sm')
from epitator.annotator import AnnoDoc
from epitator.geoname_annotator import GeonameAnnotator
from epitator.resolved_keyword_annotator import ResolvedKeywordAnnotator
from epitator.count_annotator import CountAnnotator
from epitator.date_annotator import DateAnnotator
from itertools import groupby
import datetime
import sys
import pandas as pd
import re
import numpy as np
import epitator
import numpy as np

In [191]:
import pandas as pd
import pickle
import os
from who_scraper import scrape
from my_utils import extract_from_url, flatten_list, get_edb
from annotator import *
from wiki_country_parser import get_wiki_countries_df

# WHO DF

In [105]:
path = os.path.join("pickles","parsed_who_df.p")

In [106]:
all_links = scrape(years=['2018'],proxies=None)
parsed_whos_df = pd.DataFrame.from_dict(create_annotated_database(extract_from_url(all_links),
                                                                  entity_funcs_and_params=[
                                                                      (geonames, {"raw":True}),
                                                                      (cases, {"raw":True}),
                                                                      (dates, {"raw":True}),
                                                                      keywords
                                                                  ]))

In [135]:
pickle.dump(parsed_whos_df,open(path,"wb"))

In [136]:
parsed_whos_df = pickle.load(open(path,"rb"))
parsed_whos_df.head()

,texts,dates,cases,keywords,geonames
0,Ebola virus disease – Democratic Republic of t...,"[2018-11-27, 2018-09-01, 2018-01-01, 2018-11-2...","[36, 13, 9, 6, 6, 1, 1, 2, 42, 12, 6, 6, 125, ...",[Ebola hemorrhagic fever],"[Democratic Republic of the Congo, South Sudan..."
1,Ebola virus disease – Democratic Republic of t...,"[2018-11-16, 2018-11-01, 1920-11-14, 2018-11-2...","[18, 36, 7, 6, 1, 1, 1, 5, 39, 10, 6, 4, 113, ...",[Ebola hemorrhagic fever],"[Democratic Republic of the Congo, Democratic ..."
2,Middle East respiratory syndrome coronavirus (...,"[2018-11-01, 2018-10-16, 2018-10-30, 2005-01-01]","[4, 1, 1, 2266, 804, 1]",[Middle East respiratory syndrome],"[Middle East, Kingdom of Saudi Arabia, Kingdom..."
3,Ebola virus disease – Democratic Republic of t...,"[2018-11-01, 2018-11-07, 2018-11-13, 2018-11-0...","[2, 1, 31, 4, 3, 3, 3, 31, 12, 9, 2, 103, 14, ...",[Ebola hemorrhagic fever],"[Democratic Republic of the Congo, Democratic ..."
4,Ebola virus disease – Democratic Republic of t...,"[2018-11-06, 2018-11-06, 2018-11-06, 2018-10-3...","[300, 31, 29, 15, 7, 4, 2, 2, 3, 28, 10, 88, 3...",[Ebola hemorrhagic fever],"[Democratic Republic of the Congo, Democratic ..."


# EDB Statistics

In [203]:
edb = get_edb()

/Users/aussabbood/github/nlp-surveillance/edb_clean.py:39: UserWarning: "31.09.2018" is not an existing date. Please change it
  warnings.warn('"{}" is not an existing date. Please change it'.format(date))


Süd Korea


In [208]:
edb.head()

,Zeilen-ID,Ereignis-ID,Kontinent,Ausgangs- bzw. Ausbruchsland,Sekundär betroffene Länder*,Erreger,Krankheitsbild(er),Frühestbekannter Ereignisbeginn,Erstveröffentlichung,Fälle gesamt*,...,Link zur Quelle 4,Rationale für Monitoring*,Rationale für Monitoring* Kommentar,Monitoring-Frequenz,Verbindung zu Ereignis-ID*,"RKI-Berichtsformat* (z.B. Wochenbericht, EpiLag, etc)","RKI-Berichtsformat Kategorie (z.B. Kurznachricht, Fortschreibung, etc.)",Ereignisinformation und Bewertung oder Link,Datum der letzten Aktualisierung,Protokollierung des Meetings
0,1.0,AF_Nigeria_LASV_12.03.2018,Afrika,Nigeria,Benin,Lassavirus,Lassafieber,2018-01-18,nan,1121,...,NaN,NaN,NaN,1xWöchentlich,Primärereignis,"Erlassbericht 18-02-13, PHI-Bericht 18-02-23, ...",NaN,<\\rki.local\daten\Projekte\GGBS_INIG\Bewertun...,2018-03-13,NaN
1,2.0,AF_Benin_LASV_18-03-12,NaN,Benin,nan,Lassavirus,Lassafieber,2018-01-08,nan,24,...,NaN,NaN,NaN,1xWöchentlich,AF_Nigeria_LASV_18-03-12,Nächster BMG-Wochenbericht,NaN,"Fälle aus 5 Departments: Atacora (9), Bourgou ...",2018-03-13,NaN
2,3.0,AF_Liberia_LASV_12.03.2018,NaN,Liberia,nan,Lassavirus,Lassafieber,2018-01-01,nan,42,...,NaN,NaN,NaN,1xWöchentlich,Primärereignis,Nächster BMG-Wochenbericht,NaN,"Fälle aus 3 Counties: Nimba (4), Montserrado (...",2018-03-13,NaN
3,4.0,AF_Ghana_LASV_18-03-12,NaN,Ghana,nan,Lassavirus,Lassafieber,2018-02-23,nan,1,...,NaN,NaN,NaN,1xWöchentlich,Primärereignis,Nächster BMG-Wochenbericht,NaN,26-jähriger Mann; 18-02-23 Vorstellung KH Accr...,2018-03-20,NaN
4,5.0,AS_Fiji_Men_18-03-22,NaN,Fiji,nan,Meningokokken,Meningitis,2018-03-20,nan,18,...,NaN,NaN,NaN,NaN,Primärereignis,NaN,NaN,"Zahl seit 2016 kontinuierlich ansteigend, bis ...",2018-03-22,NaN


In [209]:
def _match_country(country, look_up, translation):
    escaped = re.escape(country)
    found = list(filter(lambda x: re.findall(escaped, x), look_up))
    if len(found) == 1:
        return translation[look_up.index(found[0])]
    elif len(found) > 1:
        # Check if identity in ambiguous e.g Niger --> (Niger, Nigeria)
        identical = [found_name for found_name in found if found_name == country]
        if len(identical) == 1:
            return translation[look_up.index(identical[0])]
        else:
            return [translation[look_up.index(similar)] for similar in found]

In [245]:
korea = "Süd Korea"

In [246]:
look = get_wiki_countries_df()["state_name_de"]
look.head()

0                                  Erde
1                     Europäische Union
2      Union Südamerikanischer Nationen
3                    Afrikanische Union
4    Verband Südostasiatischer Nationen
Name: state_name_de, dtype: object

In [247]:
trans = get_wiki_countries_df()["translation_state_name"].head()

# CONTINUE HERE

In [257]:
escaped = re.escape(korea)
found = list(filter(lambda x: re.findall(escaped, x), look))
trans[look.index(str(found[0]))]

TypeError: 'RangeIndex' object is not callable

In [ ]:
if not match:
    match = _match_country(to_translate, full_state_name_de, translation)
if not match:
    match = _match_country(to_translate, translation, translation)
# If still no match
if not match:
    did_u_mean = didyoumean.didYouMean(to_translate, state_name_de)
    if did_u_mean and (did_u_mean.lower() not in continents):
        try:
            match = translate(did_u_mean)
        except RecursionError:
            print(did_u_mean)
    else:
        match = to_translate
elif isinstance(to_translate, list):
match = [translate(country) for country in to_translate]
return match

## Country statistic

In [160]:
print(len(edb)," entries", "in edb")
countries_not_null = edb[pd.notnull(edb["Ausgangs- bzw. Ausbruchsland"])]
print(len(countries_not_null)," are not empty")
unique = set(c[0] if type(c)==list else c for c in countries_not_null['Ausgangs- bzw. Ausbruchsland'].tolist())
print(len(unique), " are unique")

295  entries in edb
280  are not empty
79  are unique


In [161]:
wiki_countries_df = get_wiki_countries_df()
same_word_translated = []
for _, row in wiki_countries_df.iterrows():
    if row['state_name_de'] == row['translation_state_name']:
        same_word_translated.append(row['state_name_de'])
    elif row['full_state_name_de'] == row['translation_state_name']:
        same_word_translated.append(row['full_state_name_de'])
unprocesed_edb = pd.read_csv("edb.csv", sep=";")

# Exlude words that are unchanged compared to unproccessed version
unchanged_country_names = [e for e in unique if e in unprocesed_edb['Ausgangs- bzw. Ausbruchsland '].tolist()]
# Exlude words that are the same after translation
unchanged_country_names = [e for e in unchanged_country_names if e not in same_word_translated]
# Exlude words that were English in the first place
unchanged_country_names =  [e for e in unchanged_country_names if e not in wiki_countries_df["translation_state_name"].tolist()]
print("{} countries were not translated".format(len(unchanged_country_names)))
unchanged_country_names

7 countries were not translated


['La Reunion',
 'DRCongo',
 'Typhus',
 'Nordeuropa',
 'VAE Dubai',
 'Französisch-Polynesien',
 'Französiche Guyana']

## Direct comparison to source

In [162]:
indices_sources = list(range(15,26))
indices_sources_and_countries = indices_sources
indices_sources_and_countries.extend([3,9,11,12])
indices_sources_and_countries

[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 3, 9, 11, 12]

In [163]:
sources = edb.iloc[:,indices_sources_and_countries].copy(deep=True) # Get only the columns mentioning sources

In [164]:
mask = sources.copy(deep=True) # Create a mask for filtering
# Use only the columns about sources to filter
for column in sources.columns[:11]:
    mask[column] = sources[column].str.contains('who',na=False) # Extract all the entries that have the word "who"
    mask[column] = sources[column].str.contains('don',na=False) # and "don"
indices = [i for i in range(len(mask)) if not mask.iloc[i,:11].any()] 
sources_filtered = sources.drop(np.reshape(indices,(len(indices),)))# Drop all rows that don't mention "who" or "don"

In [165]:
# Only get the necessary information from the who dons for comparison
comparable_who = parsed_whos_df.iloc[:,[1,2,4]].copy(deep=True)

In [166]:
dates = sources_filtered.iloc[:,[1,4,7,10]].copy(deep=True)

In [167]:
dates = dates.replace("nan",np.nan)

In [168]:
dates = dates.values.tolist()
dates = [list(filter(lambda x: x==x, row)) for row in dates]
dates

[['2018-03-15'],
 ['2018-04-13'],
 ['2018-04-15', '2018-04-20', '2018-04-18'],
 ['2018-05-11', '2018-05-10'],
 ['2018-05-15', '2018-05-14'],
 ['2018-05-17', '2018-05-17'],
 ['2018-05-22', '2018-05-21'],
 ['2018-05-31'],
 ['2018-06-05', '2018-06-05'],
 ['2018-06-12'],
 ['2018-06-18'],
 ['2018-07-30', '2018-08-04'],
 ['2018-08-07', '2018-06-25'],
 ['2018-08-07'],
 ['2018-07-02'],
 ['2018-08-24'],
 ['2018-08-31']]

In [169]:
faelle = sources_filtered.iloc[:,-3:].copy(deep=True)
faelle = faelle.replace(np.nan,"nan")
faelle = faelle.values.tolist()
faelle = [list(filter(lambda x: x==x, row)) for row in faelle]
faelle

[['nan', 'nan', 'nan'],
 ['20', '19', 1.0],
 ['nan', '413', 9.0],
 ['34', '2', 'nan'],
 ['41', '2', 'nan'],
 ['44', '3', 'nan'],
 ['46', 'nan', 'nan'],
 ['nan', '15', 'nan'],
 ['16', '1', 'nan'],
 ['995', '114', 'nan'],
 ['nan', 'nan', 'nan'],
 ['nan', '16', 27.0],
 ['3', '3', 'nan'],
 ['nan', 'nan', 'nan'],
 ['nan', '1, aber 2 Nachweise bei asymptomatischen Kindern', 'nan'],
 ['1', '1', 'nan'],
 ['1', '1', 'nan']]

In [170]:
cases = []
for row in faelle:
    strings = []
    if len(row) >=1 :
        for string in row:
            string = str(string)
            found = re.findall(r"(\d{1,})",string)
            strings.extend([digit for digit in found if digit != '' and digit != "0"])
        cases.append(strings)
    else:
        cases.append(row)
cases

[[],
 ['20', '19', '1'],
 ['413', '9'],
 ['34', '2'],
 ['41', '2'],
 ['44', '3'],
 ['46'],
 ['15'],
 ['16', '1'],
 ['995', '114'],
 [],
 ['16', '27'],
 ['3', '3'],
 [],
 ['1', '2'],
 ['1', '1'],
 ['1', '1']]

In [171]:
sources_filtered.columns[-4]

'Ausgangs- bzw. Ausbruchsland'

In [172]:
comparable_edb = sources_filtered['Ausgangs- bzw. Ausbruchsland'].copy(deep=True).to_frame()
comparable_edb["cases"] = cases
comparable_edb["dates"] = dates
comparable_edb

,Ausgangs- bzw. Ausbruchsland,cases,dates
25,Oman,[],[2018-03-15]
52,Australia,"[20, 19, 1]",[2018-04-13]
70,Nigeria,"[413, 9]","[2018-04-15, 2018-04-20, 2018-04-18]"
107,"Congo, Democratic Republic of the (Kinshasa)","[34, 2]","[2018-05-11, 2018-05-10]"
112,"Congo, Democratic Republic of the (Kinshasa)","[41, 2]","[2018-05-15, 2018-05-14]"
113,"Congo, Democratic Republic of the (Kinshasa)","[44, 3]","[2018-05-17, 2018-05-17]"
120,"Congo, Democratic Republic of the (Kinshasa)",[46],"[2018-05-22, 2018-05-21]"
140,India,[15],[2018-05-31]
145,Cameroon,"[16, 1]","[2018-06-05, 2018-06-05]"
149,Brazil,"[995, 114]",[2018-06-12]


In [152]:
comparable_who.head()

,dates,cases,geonames
0,"[2018-11-27, 2018-09-01, 2018-01-01, 2018-11-2...","[36, 13, 9, 6, 6, 1, 1, 2, 42, 12, 6, 6, 125, ...","[Democratic Republic of the Congo, South Sudan..."
1,"[2018-11-16, 2018-11-01, 1920-11-14, 2018-11-2...","[18, 36, 7, 6, 1, 1, 1, 5, 39, 10, 6, 4, 113, ...","[Democratic Republic of the Congo, Democratic ..."
2,"[2018-11-01, 2018-10-16, 2018-10-30, 2005-01-01]","[4, 1, 1, 2266, 804, 1]","[Middle East, Kingdom of Saudi Arabia, Kingdom..."
3,"[2018-11-01, 2018-11-07, 2018-11-13, 2018-11-0...","[2, 1, 31, 4, 3, 3, 3, 31, 12, 9, 2, 103, 14, ...","[Democratic Republic of the Congo, Democratic ..."
4,"[2018-11-06, 2018-11-06, 2018-11-06, 2018-10-3...","[300, 31, 29, 15, 7, 4, 2, 2, 3, 28, 10, 88, 3...","[Democratic Republic of the Congo, Democratic ..."


In [ ]:
def find_matches(df1_and_indices, df2_and_indices):
    indices_matches = []
    matches = []
    df1, columns1 = df1_and_indices
    df2, columns2 = df2_and_indices
    for index , row in df1.iterrows():
        for _, row in df2.iterrows():
            if set()

In [114]:
sources_filtered.columns[-2::-3]

Index(['Datum der Veröffentlichung der Quelle 4',
       'Datum der Veröffentlichung der Quelle 3',
       'Datum der Veröffentlichung der Quelle 2',
       'Datum der Veröffentlichung der Quelle1'],
      dtype='object')

In [ ]:
# Idea which is not correct and not complete
date_matches = {}
for column in sources_filtered.columns[-2::-3]: # Use only the columns mentioning dates
    date_matches[column] = [i for i in range(len(parsed_whos_df))\
                            if list(map(lambda x: str(x)[:-3],parsed_whos_df["date"].tolist()))[i] \
                            in list(map(lambda x: str(x)[:-12],sources_filtered[column].tolist()))]

In [ ]:
import itertools
indices_that_matched = list(set(itertools.chain(*date_matches.values())))

# To present

In [ ]:
ereignisdatenbank.iloc[sources_filtered.index.tolist(),[3,6,7,9]]

In [ ]:
parsed_whos_df.iloc[indices_that_matched,1:].sort_values("date")

In [ ]:
# Prettify the link description
link_description = [re.search(r'don/(.*)/en',all_links[i])[1]\
                    .replace('-', ' ',2).replace('-',', ',2).replace('-',' ')\
                    for i in range(len(all_links))]

In [ ]:
# Extract the most important columns
compare = parsed_whos_df.iloc[:,[1,3,4]].copy()
compare['link_description'] = pd.Series(link_description,index=compare.index)

In [ ]:
compare["date"].iloc[0]

In [ ]:
# These are the links that were faulty during annotation
to_check = compare[compare["keyword"].isnull()==True].index.values
links_to_check = np.asarray(all_links)[to_check] # Get the links that caused the bad annotations

In [ ]:
annotated_faulty_text = create_annotated_database(extract(links_to_check),raw=True)

In [ ]:
faulty_df = pd.DataFrame.from_dict(annotated_faulty_text).iloc[:,1:]
faulty_df

In [ ]:
# Entries that had the occurence of a geographical entity more than once
parsed_whos_df[parsed_whos_df["geoname"].str.len()>1]

# Goodnes Test

In [ ]:
sources_filtered.head()

In [ ]:
sources_filtered = sources_filtered.fillna("nan")

In [ ]:
links_from_sources_filtered = [sources_filtered["Link zur Quelle 1"].iloc[i] 
                               if ("pdf" not in sources_filtered["Link zur Quelle 1"].iloc[i]) 
                               and (sources_filtered["Link zur Quelle 1"].iloc[i] != "nan") 
                               else sources_filtered["Link zur Quelle 2"].iloc[i] 
                               for i in range(len(sources_filtered))]

In [ ]:
extracted = extract(links_from_sources_filtered)

In [ ]:
parsed_links_ereignisdatenbank = create_annotated_database(extracted,raw=True)

In [ ]:
df = pd.DataFrame.from_dict(parsed_links_ereignisdatenbank)
df = df.iloc[:,1:]
df["links"] = links

In [ ]:
to_compare_from_ereignisdatenbank = ereignisdatenbank.iloc[sources_filtered.index.tolist()\
                                                           ,[3,6,7,8,10,11,12,13,14]].reset_index().drop("index",axis=1)

In [ ]:
# WHY DOES THIS NOT WORK
to_compare_from_ereignisdatenbank["Warcheinlische Fälle"].apply((lambda x: int(x) if not np.isnan(x) else x))

In [ ]:
extract("http://www.promedmail.org/post/5838919")

In [ ]:
comparison_df = pd.concat([df, to_compare_from_ereignisdatenbank], axis=1, sort=False)

In [ ]:
comparison_df["date"] = comparison_df["date"].astype(object) # To restore lists